# Breve Descrizione
Questo è un modello con ragionamenti praticamente analoghi all'altro classificatore con YAMnet ma allenato su un dataset molto più grande, costruito iterando le procedure per 'mixed_up_data_talk' su circa 200k record, abbastanza complesso da non far overfittare YAMNet. I risultati sono, siffatti, assolutamente più plausibili e, poi, è quello che abbiamo utilizzato nello scheduler

In [5]:
import os

# Forza utilizzo della CPU per il train, più 'sbrigativo' che CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
import numpy as np
import librosa
import tensorflow as tf

tf.config.optimizer.set_jit(False)

import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from joblib import load

ModuleNotFoundError: No module named 'pandas'

In [1]:
CSV_PATH   = "datasets/labels.csv"
MODEL_OUT  = "yamnet_model_large_dataset.joblib"      # Salvo nella directory corrente, si può cambiare percorso
SR         = 16000                      


df = pd.read_csv(CSV_PATH)
label_map = {"music": 0, "noisy": 1}
df["label_num"] = df["label"].map(label_map)

file_label_list = list(zip(df["filepath"].tolist(),
                           df["label_num"].tolist()))

yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

def extract_yamnet_embedding(wav_path, sr=16000):
    wav, _ = librosa.load(wav_path, sr=sr, mono=True)

    wav_tf = tf.convert_to_tensor(wav, dtype=tf.float32)

    scores, embeddings, spectrogram = yamnet_model(wav_tf)

    emb = tf.reduce_mean(embeddings, axis=0)
    return emb.numpy()


/Users/domenicodemarchis/Desktop/IAARTS/classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = []
y = []
for path, label in file_label_list:
    emb = extract_yamnet_embedding(path)
    X.append(emb)
    y.append(label)
X = np.vstack(X)
y = np.array(y)

scaler = StandardScaler().fit(X)

norm = scaler.fit_transform(X)

lda = LinearDiscriminantAnalysis()
X_reduced = lda.fit_transform(norm,y)

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
clf = LogisticRegression(
    max_iter=500,
    class_weight='balanced',
    random_state=42
)
clf.fit(X_train, y_train)


print("Valutazione su Test Set:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['music','noisy']))
print("Matrice di Confusione:\n", confusion_matrix(y_test, y_pred))


print(f"Modello YAMNet salvato in {MODEL_OUT} …")
joblib.dump({'pca': lda, 'clf': clf}, MODEL_OUT)

def predict_noisy_probability(wav_path):

    emb = extract_yamnet_embedding(wav_path).reshape(1, -1)
    emb_scaled = scaler.transform(emb)         
    emb_lda    = lda.transform(emb_scaled)    
    prob_noisy = clf.predict_proba(emb_lda)[0, 1]
    return prob_noisy


Valutazione su Test Set:
              precision    recall  f1-score   support

       music       0.75      0.62      0.68       706
       noisy       0.63      0.76      0.69       600

    accuracy                           0.69      1306
   macro avg       0.69      0.69      0.69      1306
weighted avg       0.70      0.69      0.69      1306

Matrice di Confusione:
 [[441 265]
 [146 454]]
Modello YAMNet salvato in yamnet_head.joblib …


In [1]:
test_files = [
    "audio_test/voice_base_music.wav",
    "audio_test/voice_base_pure.wav",
    "audio_test/voice_base_noise.wav"
    ]

data = load("yamnet_model_large_dataset.joblib")
pca = data['pca']
clf = data['clf']

for audio_file in test_files:
    waveform, sr = librosa.load(audio_file, sr=16000, mono=True)

    yamnet = hub.load("https://tfhub.dev/google/yamnet/1")
    _, embeddings, _ = yamnet(waveform)
    feat = np.mean(embeddings.numpy(), axis=0).reshape(1, -1)

    feat_pca = pca.transform(feat)

    pred_label = clf.predict(feat_pca)
    pred_proba = clf.predict_proba(feat_pca)

    base, _ = os.path.splitext(audio_file)
    print(f"Predizione per {base}:", pred_label[0])
    print(f"Probabilità per {base}:", pred_proba[0])
    print("\n")

NameError: name 'load' is not defined